In [122]:
import numpy as np
import pandas as pd
import scipy as sc
from scipy import stats
from matplotlib import pyplot as plt
from statsmodels.sandbox.stats.multicomp import multipletests
import statsmodels.stats.multitest as smm

Описание используемых данных
Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

Практическая значимость изменения
Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). 


In [70]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [71]:
data.describe()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,...,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,2.463922,7.100958,19.056151,15.618688,23.537120,11.303466,8.921396,63.270147,53.906324,70.678573,...,5.563444,2.192029,1.967424,2.181360,1.729308,1.980733,16.834075,1.825827,2.287380,1.756827
std,1.413687,4.164703,9.287753,4.664146,4.475294,4.055122,3.270713,13.792140,45.635498,21.326237,...,2.389789,2.454218,2.047129,2.542652,1.507149,2.230157,7.126802,1.902716,3.185571,1.623151
min,0.833898,1.231043,4.941211,6.395270,8.520713,2.066576,1.364917,42.544976,24.616767,42.631422,...,1.144240,0.853957,0.853957,0.853957,0.833898,0.833898,4.675683,0.833898,0.853957,0.833898
25%,1.284642,3.892403,13.423946,12.574596,21.151624,8.712898,6.213396,53.093627,34.009687,54.202100,...,3.805875,1.042783,1.011350,1.011350,1.003337,1.003337,11.186633,1.003337,1.011350,1.003337
50%,2.104677,6.315551,16.734855,14.586150,23.477006,11.172590,8.703397,63.230911,39.554954,65.510651,...,5.617824,1.339507,1.273861,1.298543,1.252527,1.252527,16.514389,1.252527,1.273861,1.252527
75%,3.345067,8.916399,21.884325,17.695678,26.471909,13.568625,11.139240,69.880705,50.295756,79.427207,...,7.100942,1.815754,1.647255,1.677971,1.625364,1.611299,21.849385,1.607345,1.647255,1.611299
max,7.364879,20.006038,60.584449,29.659104,34.110743,27.441093,17.121366,132.144503,327.590426,128.606260,...,11.227770,11.913855,10.883310,11.692697,10.392539,10.640754,49.295538,9.919132,17.278985,9.333904


,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [12]:
data.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)
для групп early neoplasia (control) и cancer (treatment)
В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [130]:

normal = data.loc[data.Diagnosis == 'normal'].iloc[:,2:]
earneop = data.loc[data.Diagnosis == 'early neoplasia'].iloc[:,2:]
cancer = data.loc[data.Diagnosis == 'cancer'].iloc[:,2:]


,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
49,1.618129,4.854387,7.646980,11.631036,19.839961,19.484855,15.492407,78.090449,49.035810,71.704607,...,7.175936,1.618129,1.618129,1.618129,1.618129,1.618129,22.008936,1.618129,1.618129,1.618129
50,1.609022,10.632542,28.344988,18.462903,23.702379,13.494061,6.091660,58.775216,88.026904,74.055824,...,6.091660,1.609022,1.609022,1.609022,1.609022,1.609022,24.840164,1.609022,1.609022,1.609022
51,1.549953,3.900629,12.496120,10.242207,22.972095,16.860563,7.324792,76.410218,109.988431,102.073552,...,6.873594,1.549953,1.549953,1.549953,1.549953,1.549953,18.834745,1.549953,1.549953,1.549953
52,1.606786,4.043656,8.849487,15.158388,26.287855,27.441093,8.033929,132.144503,188.797910,120.965118,...,1.606786,1.606786,1.606786,1.606786,1.606786,1.606786,21.215253,1.606786,1.606786,1.606786
53,2.697819,8.225604,21.299676,14.114099,22.289155,14.341455,5.567752,59.058349,103.291207,53.138337,...,7.099892,0.899273,0.899273,0.899273,0.899273,0.899273,20.368088,0.899273,0.899273,0.899273
54,1.126738,4.996760,13.442388,11.691325,21.169493,8.703401,6.729059,58.340683,135.862727,53.148145,...,7.214646,1.126738,1.126738,1.126738,1.126738,1.126738,8.098943,1.126738,1.126738,1.126738
55,3.163299,11.208777,25.033996,14.749504,28.573643,18.891720,12.813277,45.045398,27.944133,53.444257,...,5.546224,1.054433,1.054433,1.054433,1.054433,1.054433,28.521715,1.054433,1.054433,1.054433
56,1.428276,6.749770,14.635476,14.066889,22.172809,14.258937,13.872182,45.667628,63.205591,70.566094,...,4.284829,1.428276,1.428276,1.428276,1.428276,1.428276,11.515131,1.428276,1.428276,1.428276
57,2.604658,7.045005,23.623998,13.348245,25.032975,11.995723,6.408005,49.690114,57.398979,60.452996,...,7.813974,1.034986,1.034986,1.034986,1.034986,1.034986,13.132502,1.034986,1.034986,1.034986
58,3.331313,7.558592,25.346209,17.142989,30.953137,13.123224,10.317712,70.472701,54.712768,62.904109,...,4.578452,1.110438,1.110438,1.110438,1.110438,1.110438,11.804114,1.110438,1.110438,1.110438


In [156]:
alpha = 0.05
ttest_normal_earneop = stats.ttest_ind(normal, earneop, equal_var = False).pvalue
ttest_earneop_cancer = stats.ttest_ind(earneop, cancer, equal_var = False).pvalue
ttest_normal_earneop_stat_val = []
ttest_earneop_cancer_stat_val = []
for i in ttest_earneop_cancer:
    if i < 0.05: ttest_earneop_cancer_stat_val.append(i)
for i in ttest_normal_earneop:
    if i < 0.05: ttest_normal_earneop_stat_val.append(i)
len(ttest_normal_earneop_stat_val), len(ttest_earneop_cancer_stat_val)

(1575, 3490)

Часть 2: поправка методом Холма¶
Для этой части задания нам понадобится модуль multitest из statsmodels.

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратим внимание, что поскольку мы будем делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитать для каждого значимого изменения fold change и выписать в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

Обратим внимание, что

применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия;
при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

In [184]:
def fold_change(C, T):
    if T > C: return T/C
    else: -C/T

(1575, 3490)

In [190]:
alpha = 0.05
normal_earneop_rejected, normal_earneop_p_corr, _, _ = smm.multipletests(ttest_normal_earneop_stat_val, alpha=alpha/2, method='holm')
earneop_cancer_rejected, earneop_cancer_p_corr, _, _ = smm.multipletests(ttest_earneop_cancer_stat_val, alpha=alpha/2, method='holm')
normal_earneop_val_p = [p[0] for p in np.argwhere(normal_earneop_p_corr < alpha / 2)]
earneop_cancer_val_p = [p[0] for p in np.argwhere(earneop_cancer_p_corr < alpha / 2)]


(5, 128)

In [199]:
real_norm_earneop_val_p = [gene_num for gene_num in normal_earneop_val_p 
              if np.abs(fold_change(normal.iloc[:, gene_num].mean(), earneop.iloc[:, gene_num].mean())) > 1.5]
real_earneop_cancer_val_p = [gene_num for gene_num in earneop_cancer_val_p 
              if np.abs(fold_change(earneop.iloc[:, gene_num].mean(), cancer.iloc[:, gene_num].mean())) > 1.5]
len(real_norm_earneop_val_p), len(real_earneop_cancer_val_p)

(0, 11)


Часть 3: поправка методом Бенджамини-Хохберга
Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратим внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H0, когда они есть, и будут чаще отклонять H0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [205]:
rej1, earneop_cancer_p_corr_benj, _, _ = smm.multipletests(ttest_earneop_cancer_stat_val, alpha=alpha/2, method='fdr_bh')
rej2, normal_earneop_p_corr_benj, _, _ = smm.multipletests(ttest_normal_earneop_stat_val, alpha=alpha/2, method='fdr_bh')
normal_earneop_val_p_benj = [p[0] for p in np.argwhere(normal_earneop_p_corr_benj < alpha / 2)]
earneop_cancer_val_p_benj = [p[0] for p in np.argwhere(earneop_cancer_p_corr_benj < alpha / 2)]
len(normal_earneop_val_p_benj), len(earneop_cancer_val_p_benj)

(533, 2187)

In [206]:
real_norm_earneop_val_p_benj = [gene_num for gene_num in normal_earneop_val_p_benj 
              if np.abs(fold_change(normal.iloc[:, gene_num].mean(), earneop.iloc[:, gene_num].mean())) > 1.5]
real_earneop_cancer_val_p_benj = [gene_num for gene_num in earneop_cancer_val_p_benj 
              if np.abs(fold_change(earneop.iloc[:, gene_num].mean(), cancer.iloc[:, gene_num].mean())) > 1.5]
len(real_norm_earneop_val_p_benj), len(real_earneop_cancer_val_p_benj)

(11, 154)